# Environmental Master Code
## This is my master code for the Raspberry Pi.
##### Please do not edit, copy, or use without permission from owner, thank you.

### Import coding libraries to use

In [4]:
import busio ##black sensor library
import board ##black sensor library
import adafruit_ccs811 ##black sensor library
import smbus ##blue sensor libarary
import gspread ##google sheets library
import time ##library for general use

### Code for Google Sheets

In [5]:
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "EnviroPiLog"

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1

### Code for the black external sensor

In [6]:
from board import *
i2c = board.I2C()   # uses board.SCL and board.SDA

In [7]:
#black sensor talks with adafruit
ccs =  adafruit_ccs811.CCS811(i2c)

In [8]:
#print black sensor readings (pause after you get a reading to continue)
while True:
    print("CO2: ", ccs.eco2, " TVOC:", ccs.tvoc)
    print(ccs.data_ready)
    time.sleep(3)

CO2:  400  TVOC: 0
False


KeyboardInterrupt: 

### Code for the blue external sensor

In [9]:
#read blue sensor data and calculate the external temperature
def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [10]:
#contact and sign into Adafruit with the .env
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME")
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

In [11]:
#talk with bme280
from bme280 import BME280
bme280 = BME280()

### Main code for everything

In [19]:
#read adafruit feeds
exterFeed = aio.feeds("external")
tempFeed = aio.feeds("temperature")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
co2Feed = aio.feeds("co2")
tvocFeed = aio.feeds("tvoc")

#create metadata (location)
metadata = {"lat": 40.262726,
            "lon": -74.94198,
            "ele": 0,
            "created_at": None}

#google sheets import datetime
from datetime import datetime

#MAIN LOOP
while True:
    #create letter variables
    e = readTemp()
    t = bme280.get_temperature()
    f = e * (9./5.) +32
    q = t * (9./5.) +32 
    h = bme280.get_humidity()
    p = bme280.get_pressure()
    co2 = ccs.eco2
    tvoc = ccs.tvoc
    #talk with aio.send_data
    aio.send_data(exterFeed.key, f, metadata)
    aio.send_data(tempFeed.key, q, metadata)
    aio.send_data(humidFeed.key, h, metadata)
    aio.send_data(pressFeed.key, p, metadata)
    aio.send_data(co2Feed.key, co2, metadata)
    aio.send_data(tvocFeed.key, tvoc, metadata)
    #print data to google sheets
    now=datetime.now()
    now_time = now.strftime("%H:%M:%S")
    now_day = now.strftime("%m/%d/%Y")
    row=[now_day, now_time, f, q, h, p, co2, tvoc]
    sheet.append_row(row)
    #print data to notebook
    print(f, ":°Fahrenheit|")
    print(q, ":°Fahrenheit|")
    print(h, ":Humidity|")
    print(p, ":Barometric Pressure|")
    print(co2, ": PPM (CO2)|")
    print(tvoc, ":PPB (TVOC)|")
    print()
    #time in between the intervals the Pi takes data (fifteen seconds for me)
    time.sleep(15)

APIError: {'code': 400, 'message': "Unable to parse range: 'Sheet1'", 'status': 'INVALID_ARGUMENT'}